In [1]:
from utils.utils import PNode
import pandas as pd

df_sovanta = pd.read_csv("../eval_sovanta/eval-retrieval-meta--llama3.1-70b-instruct_context_relevance_with_cot.csv")
df_wikieval = pd.read_csv("../eval_wikieval/eval-retrieval-gpt-4o_context_relevance_with_cot-2025-06-09_18_52_33.csv")

df_sovanta["retrieved_text"] = df_sovanta.apply(
    lambda row: " ".join([n.text for n in pd.eval(row["nodes"], local_dict={"PNode": PNode})]),
    axis=1,
)
df_wikieval["retrieved_text"] = df_wikieval.apply(
    lambda row: " ".join([n.text for n in pd.eval(row["nodes"], local_dict={"PNode": PNode})]),
    axis=1,
)

# Wikieval

In [2]:
print(
    "Correlation: ",
    df_wikieval["nodes_recall_0.5"].corr(df_wikieval["gpt-4o_context_relevance_with_cot"]),
)

df_wikieval = df_wikieval[
    [
        "chunk_size",
        "top_k",
        "rerank_model",
        "prompt",
        "relevant_text",
        "retrieved_text",
        "nodes_recall_0.5",
        "gpt-4o_context_relevance_with_cot",
    ]
]
df_wikieval["abs_diff"] = (df_wikieval["nodes_recall_0.5"] - df_wikieval["gpt-4o_context_relevance_with_cot"]).abs()
highlighted_df = df_wikieval[df_wikieval["abs_diff"] > 0.5]
highlighted_df

Correlation:  0.46489139878467295


,chunk_size,top_k,rerank_model,prompt,relevant_text,retrieved_text,nodes_recall_0.5,gpt-4o_context_relevance_with_cot,abs_diff
1,64,2,NaN,What is the objective of the Uzbekistan-Afghan...,"[""The Uzbekistan–Afghanistan–Pakistan Railway ...",passing through Afghanistan's territory. This ...,0.387755,1.000000,0.612245
4,64,2,NaN,What is theranostics and how does it combine d...,"['Theranostics, also known as theragnostics, i...",Theranostics encompasses a range of therapeuti...,0.105871,1.000000,0.894129
15,64,2,NaN,When did Pope Benedict XVI become the head of ...,"['Pope Benedict XVI (Latin: Benedictus PP.', '...",was the head of the Catholic Church and sovere...,0.337438,1.000000,0.662562
17,64,2,NaN,How does the height and thrust of the Starship...,"[""At 120 metres (394 feet) in height and with ...",At 120 metres (394 feet) in height and with a ...,0.069079,0.666667,0.597588
18,64,2,NaN,What is the Kyzylkum Desert known for in terms...,['The desert is well known for its deposits of...,The local population uses the large spaces of ...,0.155172,0.666667,0.511494
...,...,...,...,...,...,...,...,...,...
8531,512,2,mixedbread-ai/mxbai-rerank-base-v1,"How long did the Siege of Mariupol last, and w...",['The Siege of Mariupol began on 24 February 2...,all Ukrainian troops remaining in the city sur...,0.399664,1.000000,0.600336
8554,512,2,mixedbread-ai/mxbai-rerank-xsmall-v1,What is theranostics and how does it combine d...,"['Theranostics, also known as theragnostics, i...",PET scans can also guide treatment decisions b...,0.285714,1.000000,0.714286
8581,512,2,mixedbread-ai/mxbai-rerank-xsmall-v1,"How long did the Siege of Mariupol last, and w...",['The Siege of Mariupol began on 24 February 2...,all Ukrainian troops remaining in the city sur...,0.399664,1.000000,0.600336
8615,512,4,NaN,When did Pope Benedict XVI become the head of ...,"['Pope Benedict XVI (Latin: Benedictus PP.', '...",was the head of the Catholic Church and sovere...,0.275862,1.000000,0.724138


In [12]:
example_case = highlighted_df.iloc[7]
print("Prompt: ", example_case["prompt"])
print("-----------------------------\nRelevant Text:\n", example_case["relevant_text"])
print("-----------------------------\nRetrieved Text:\n", example_case["retrieved_text"])

Prompt:  What was the estimated timeline for fully restoring power in Moore County after the shooting attack on the electrical distribution substations?
-----------------------------
Relevant Text:
 ['Initial estimates were that up to four days could be required to fully restore power in the area.']
-----------------------------
Retrieved Text:
 On December 3, 2022, a shooting attack was carried out on two electrical distribution substations located in Moore County, North Carolina, United States. Damage from the attack left up to 40,000 residential and business customers without electrical power. As of December 6, it was estimated that about 35,000 Moore County residents were still without power, and the timeline for completing repairs and restoring power county-wide was revised from December 8 to midnight December 7. By the morning of December 7, the number of affected residents without power was down to about 23,000,


Observations:
- Misjudgement occurs almost only with very short context sizes
- Misjudgement is characterized by context relevance being 1 and recall being low

Error classes:
- 1: Retrieved text actually contains relevant information, but relevant text contains additional information that is technically not necessary to answer the question
- 2: Important information missing from the retrieved text, but text is still relevant enough to score high context relevance

# sovanta

In [3]:
print(
    "Correlation: ",
    df_sovanta["nodes_recall_0.5"].corr(df_sovanta["meta--llama3.1-70b-instruct_context_relevance_with_cot"]),
)

df_sovanta = df_sovanta[
    [
        "chunk_size",
        "top_k",
        "rerank_model",
        "prompt",
        "relevant_text",
        "retrieved_text",
        "nodes_recall_0.5",
        "meta--llama3.1-70b-instruct_context_relevance_with_cot",
    ]
]
df_sovanta["ctx"] = df_sovanta["chunk_size"] * df_sovanta["top_k"]
df_sovanta["abs_diff"] = (
    df_sovanta["nodes_recall_0.5"] - df_sovanta["meta--llama3.1-70b-instruct_context_relevance_with_cot"]
).abs()
highlighted_df = df_sovanta[df_sovanta["abs_diff"] > 0.5]

highlighted_df

Correlation:  0.23114596970225915


,chunk_size,top_k,rerank_model,prompt,relevant_text,retrieved_text,nodes_recall_0.5,meta--llama3.1-70b-instruct_context_relevance_with_cot,ctx,abs_diff
0,64,2,NaN,Was ist das GenAI Starter?,['Mit unserem GenAI Starterpaket können Sie di...,Kickstart your company‘s innovation with Gener...,0.000000,1.000000,128,1.000000
1,64,2,NaN,What is the GenAI lab?,"[""With our GenAI starter package, you can star...",How do you ensure that hallucinations of LLMs ...,0.000000,0.666667,128,0.666667
4,64,2,NaN,Was sind die wichtigsten development guidelines,['The sovanta Development Standards are a set ...,The development standards is a living document...,0.000000,0.666667,128,0.666667
5,64,2,NaN,Was ist Experience Managemenet,"['experience management**, a broad discipline ...",The XM team is led by [Andrew White](https://s...,0.000000,0.666667,128,0.666667
7,64,2,NaN,Was findest du zu LLMs?,['LLMs are a type of AI modelthat specializes ...,or LLMs. These are specially designed machine\...,0.136015,0.666667,128,0.530651
...,...,...,...,...,...,...,...,...,...,...
19456,512,14,mixedbread-ai/mxbai-rerank-xsmall-v1,Was sind die top ziele der mainova,"['spätestens bis 2040 klimaneutral', 'Strategi...",ob diese eingehalten werden.\nIm Zusammenhang ...,0.393797,1.000000,7168,0.606203
19460,512,14,mixedbread-ai/mxbai-rerank-xsmall-v1,why cant i create a new project,['if you get the hint here that no project was...,# Content: Create a new Project\n1. Click on “...,1.000000,0.333333,7168,0.666667
19470,512,14,mixedbread-ai/mxbai-rerank-xsmall-v1,Fasse bitte mit den Infos im Dokument Stichpu...,"['From 7.500 €*(4h-Workshop, onsite or remote,...","![Ein Bild, das Screenshot, Dunkelheit, Schwar...",1.000000,0.000000,7168,1.000000
19478,512,14,mixedbread-ai/mxbai-rerank-xsmall-v1,development stages,"['1. performance', 'consider performance while...",Duration: 3h\nLocation: Remote\nThis crucial a...,0.270556,1.000000,7168,0.729444


In [9]:
example_case = highlighted_df.iloc[8]
print("Prompt: ", example_case["prompt"])
print("-----------------------------\nRelevant Text:\n", example_case["relevant_text"])
print("-----------------------------\nRetrieved Text:\n", example_case["retrieved_text"])

Prompt:  Wie fülle ich als STudent meinen Stundenzettel aus?
-----------------------------
Relevant Text:
 ['| Tag | Beginn | Ende | Arbeitszeit \\n(in h) | gesetzl. Mindest-pause (in h) | eingehaltene Pause (in h) | Abwesenheits-art |', 'pleasealsoenter “Urlaub (Leave)” or “Krankheit (Sickness)” in the last column of the time sheet accordingly.']
-----------------------------
Retrieved Text:
 You can find the time sheet on the SharePoint →
Template_Stundenzettel. Template_Stundenzettel. Your monthly salary is calculated on the basis of this.


Observations:
- Misjudgement occurs across the board, but is a little more prevelant with smaller context sizes
- Misjudgement is characterized by context relevance being 1 and recall being low, or recall being 1 and context relevance lower

Error classes:
- 1: Retrieved text is slightly different from relevant text (e.g. different language), but has very similar meaning and therefore relevance
- 2: Important information missing from the retrieved text, but text is still relevant enough to score high context relevance
- 3: Retrieved text is relevant to the query, but does not contain the information that is asked for